##### Pacotes Necessários 

In [94]:
import psycopg2
import pandas as pd
from configparser import ConfigParser

##### Conexão com a Base e Extração de Dados

In [95]:
# Função para configuração de conexão com parâmetros de acesso presentes no arquivo database.ini
# Transforma o arquivo em uma variável de objeto
def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Seção {0} não encontrada no arquivo {1}'.format(section, filename))
    return db

In [98]:
def connect():
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()

        retorno = []

        cursor.execute('SELECT * FROM forms')
        forms = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'forms'")
        colunas = cursor.fetchall()
        retorno.append([forms, colunas])

        cursor.execute('SELECT * FROM orders')
        orders = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders'")
        colunas = cursor.fetchall()
        retorno.append([orders, colunas])

        cursor.execute('SELECT * FROM order_items')
        items = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'order_items'")
        colunas = cursor.fetchall()
        retorno.append([items, colunas])

        cursor.close()
        return retorno
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Conexão encerrada.')


if __name__ == '__main__':
    retorno = connect()

Conexão encerrada.


In [122]:
forms = pd.DataFrame(retorno[0][0])
forms.columns = [item for sublist in retorno[0][1] for item in sublist]

orders = pd.DataFrame(retorno[1][0])
orders.columns = [item for sublist in retorno[1][1] for item in sublist]

order_items = pd.DataFrame(retorno[2][0])
order_items.columns = [item for sublist in retorno[2][1] for item in sublist]

##### Manipulação de dados

In [123]:
# Remoção de colunas
forms = forms.drop(['id_cliente','data','sistema_operacional','navegador','localizacao','locale','cidade','utm_source',
'caracteristica','efeitos_desejados','tempo_procedimento'], axis = 1)

orders = orders[['id_pedido','target']]

order_items = order_items[['id_pedido','id_form']]

In [133]:
# Selecionando apenas os valores únicos da coluna 'id_form' da tabela 'order_items'
items_form = order_items.drop_duplicates(subset = ['id_form'])
# Left join da tabela 'forms' com a 'items_form' seguido de left join da tabela resultante com a tabela 'orders'. 
# df é a base de dados resultante
df = forms.merge(items_form, 
                 on = 'id_form', 
                 how = 'left').merge(orders, 
                                     on = 'id_pedido', 
                                     how = 'left').drop(['id_form','id_pedido'], axis = 1)

In [140]:
# Lidando com NAs
df.target.fillna(0, inplace = True)
df.dropna(axis = 0, inplace = True)


In [176]:
# Transformação de variáveis
df.estado = pd.factorize(df.estado)[0]
df.tipo_cabelo = pd.factorize(df.tipo_cabelo)[0]

,estado,qtde_lavagem_semana,tipo_cabelo,comprimento,tipo_fios,procedimentos,tempo_procedimento,dieta,atividade_fisica,frequencia_estresse,faixa_etaria,efeitos_desejados,fragancia,target
0,0,4,Tipo 2,Raíz exposta,Pontas duplas,Nenhum,Há 2 semanas,Celíaca,Frequente,Nunca,25 a 34,Efeito 7,Rosas,1.0
1,0,7,Tipo 4,Muito curto,Perfeitos,Todos acima,Há mais de 1 mês,Nenhuma restrição,Nunca,Nunca,55 a 64,Efeito 1,Eucalipto,0.0
2,0,2,Tipo 1,Extra longo,Rebeldes,Todos acima,Há mais de 1 mês,Nenhuma restrição,Raramente,Regularmente,2 a 10,Efeito 7,Rosas,0.0
3,1,5,Tipo 1,Extra longo,Rebeldes,Todos acima,Há mais de 1 mês,Celíaca,Raramente,Regularmente,25 a 34,Efeito 7,Rosas,0.0
4,2,4,Tipo 3,Muito curto,Rebeldes,Tintura,Há 1 semana,Celíaca,Sempre,Nunca,18 a 24,Efeito 7,Eucalipto,0.0


In [109]:
teste3 = teste2.drop(['id_form','id_cliente_x','data','navegador','localizacao','locale_x','cidade_x', 'utm_source_x',
'id','id_pedido', 'id_produto','qtde','valor','id_cliente_y','data_pedido','status_pagamento','tipo_pagamento',
 'valor_total','valor_desconto','cidade_y','estado_y','locale_y','utm_source_y'], axis = 1)

In [110]:
teste3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427950 entries, 0 to 427949
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   sistema_operacional  427950 non-null  object 
 1   estado_x             427752 non-null  object 
 2   qtde_lavagem_semana  427950 non-null  int64  
 3   tipo_cabelo          427950 non-null  object 
 4   comprimento          427950 non-null  object 
 5   tipo_fios            427950 non-null  object 
 6   procedimentos        427950 non-null  object 
 7   tempo_procedimento   427950 non-null  object 
 8   dieta                427950 non-null  object 
 9   atividade_fisica     427950 non-null  object 
 10  frequencia_estresse  427950 non-null  object 
 11  caracteristica       427950 non-null  object 
 12  faixa_etaria         427950 non-null  object 
 13  efeitos_desejados    426971 non-null  object 
 14  fragancia            427950 non-null  object 
 15  frete_gratis     